In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) #Connect colab to drive where whole project structure is stored.

Mounted at /content/drive


In [ ]:
!	pip install ultralytics --quiet #Yolo library
! pip install mlflow --quiet
! pip install codecarbon --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.8/179.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
import mlflow
import os

#Connecting to MLFlow
os.environ['MLFLOW_TRACKING_USERNAME'] = input('Enter your DAGsHub username: ') #fe. LouisVanLangendonck
#os.environ['MLFLOW_TRACKING_USERNAME'] = 'LouisVanLangendonck'
os.environ['MLFLOW_TRACKING_PASSWORD'] = input('Enter your DAGsHub access token: ') #copy from dagshub
# os.environ['MLFLOW_TRACKING_PASSWORD'] =
os.environ['MLFLOW_TRACKING_URI'] = input('Enter your DAGsHub project tracking URI: ') #https://dagshub.com/Sebastianpaglia/MLOps_WhereIsWally.mlflow
#os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Sebastianpaglia/MLOps_WhereIsWally.mlflow'

#Make Connection
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])

In [ ]:
from ultralytics import YOLO
from codecarbon import EmissionsTracker
import pandas as pd


emissions_output_folder = '/content/drive/MyDrive/FIB-2023-2024/MLOps/metrics'
models_output_folder = '/content/drive/MyDrive/FIB-2023-2024/MLOps/models'

# Load the model.
model = YOLO('yolov8m.pt') #Load YOLO model

# Training.
mlflow.set_experiment("training_medium_yolo_v8_SGD") #Set experiment Name
with mlflow.start_run(run_name="yolo_v8"):
  with EmissionsTracker(                        #Track emission in CSV folder
        output_dir=emissions_output_folder,
        output_file="emissions.csv",
        on_csv_write="update",
    ):
      results = model.train(
        data='/content/drive/MyDrive/FIB-2023-2024/MLOps/dataset-YOLOv8/data.yaml',
        resume=True,
        imgsz=640,
        epochs=20,
        lr0=0.1,
        lrf=0.001,
        batch=16,
        optimizer='SGD',
        name='yolov8m_SGD')

  # Log the CO2 emissions to MLflow
  emissions = pd.read_csv(emissions_output_folder + "//emissions.csv")
  emissions_metrics = emissions.iloc[-1, 4:13].to_dict()
  emissions_params = emissions.iloc[-1, 13:].to_dict()
  mlflow.log_params(emissions_params)   #Store emission in MLFlow experiment
  mlflow.log_metrics(emissions_metrics)


[codecarbon INFO @ 13:16:11] [setup] RAM Tracking...
[codecarbon INFO @ 13:16:11] [setup] GPU Tracking...
[codecarbon INFO @ 13:16:11] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:16:11] [setup] CPU Tracking...
[codecarbon WARNING @ 13:16:11] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:16:13] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 13:16:13] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 13:16:13] >>> Tracker's metadata:
[codecarbon INFO @ 13:16:13]   Platform system: Linux-5.15.120+-x86_64-with-glibc2.35
[codecarbon INFO @ 13:16:13]   Python version: 3.10.12
[codecarbon INFO @ 13:16:13]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 13:16:13]   Available RAM : 12.678 GB
[codecarbon INFO @ 13:16:13]   CPU count: 2
[codecarbon INFO @ 13:16:13]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 13:16:13]   GPU

SyntaxError: ignored